In [1]:
import numpy as np
import xarray as xr
import os
import glob
from pathlib import Path 
import pathlib
from pprint import pprint
import matplotlib.pyplot as plt
import time
import requests
import s3fs
from os.path import dirname, join
import datetime
import myDate
import netCDF4

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
warnings.filterwarnings('ignore')

In [2]:
alongtrack_file_dir = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_nosicapplied/')
alongtrack_file_dir_randomnoise = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise_nosicapplied/')
alongtrack_file_dir_missingdata = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_missingdata_nosicapplied/')
alongtrack_file_dir_orbiterror_2cm = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_2cm_nosicapplied/')
alongtrack_file_dir_3errors_2cm = Path('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_2cm_nosicapplied/')
grids_file_dir = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_nosicapplied/')
grids_file_dir_randomnoise = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_randomnoise_nosicapplied/')
grids_file_dir_missingdata = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_missingdata_nosicapplied/')
grids_file_dir_orbiterror_2cm = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_orbiterror_2cm_nosicapplied/')
grids_file_dir_3errors_2cm = Path('/home/jovyan/Data/ECCO_GMSL/gridding_output_3errors_2cm_nosicapplied/')
main_dir = Path('/home/jovyan/ECCO_GMSL/')
output_dir = Path('/home/jovyan/Data/ECCO_GMSL/GMSL/')

# Compute ECCO along track GMSL with simple weights

### No error
#### Compute the reference global mean SSH over 20 years 1996-2016

In [3]:
# ECCO_alongtrack_files=[]
# for year in range(1996,2017):
#     ECCO_alongtrack_files_tmp = list(alongtrack_file_dir.glob('*ECCO_V4r4_alongtrack_SSH_*'+str(year)+'*nc'))
#     ECCO_alongtrack_files.extend(ECCO_alongtrack_files_tmp)
# print(np.sort(ECCO_alongtrack_files))

In [4]:
# alongtrack = xr.open_mfdataset(
#         paths=ECCO_alongtrack_files,
#         combine='nested',
#         concat_dim='i',
#         decode_cf=True,
#         compat='override',
#         coords='minimal',
#         chunks={'time': 1}  
#     )  
# alongtrack

In [5]:
# ref_mean=alongtrack.SSH_at_xy.mean(skipna=True).values
# print(ref_mean)

#### Compute GMSL 10-day time series

In [6]:
ECCO_alongtrack_files = list(alongtrack_file_dir.glob('*ECCO_V4r4_alongtrack_SSH*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
# print(ECCO_alongtrack_files)

In [7]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    w=np.cos(np.deg2rad(alongtrack.lat))
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
    
    # fig = plt.figure(figsize=(15,5))
    # cm = plt.get_cmap('gist_rainbow')
    # ax=fig.gca()
    # ax = plt.axes(projection=ccrs.Robinson( \
    #               central_longitude=-67, globe=None))
    # ax.gridlines()
    # ax.add_feature(cfeature.LAND)
    # ax.add_feature(cfeature.COASTLINE)
    # kk=12
    # p=ax.scatter(alongtrack.lon[::kk],\
    #                      alongtrack.lat[::kk], \
    #                      c=(SSH_at_xy_ano*w)[::kk], s=1,\
    #                      transform=ccrs.PlateCarree(),
    #                      vmin=-1,vmax=1, cmap='jet')
alongtrack_gmsl = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [8]:
SSH_at_xy_ano

<xarray.DataArray 'SSH_at_xy' (i: 59523)>
dask.array<where, shape=(59523,), dtype=float32, chunksize=(59523,), chunktype=numpy.ndarray>
Coordinates:
    time        datetime64[ns] 2017-12-31
    cycle_day   int64 0
    cycle       int64 924
    delta_days  int64 9230
    lon         (i) float32 dask.array<chunksize=(59523,), meta=np.ndarray>
    lat         (i) float32 dask.array<chunksize=(59523,), meta=np.ndarray>
Dimensions without coordinates: i
Attributes:
    source:   ECCO V4r4

In [9]:
alongtrack_gmsl=alongtrack_gmsl.drop('cycle_day')
alongtrack_gmsl=alongtrack_gmsl.drop('delta_days')
alongtrack_gmsl.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl.attrs['units'] = 'm'
alongtrack_gmsl.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data'#, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl.time.attrs['long_name'] = 'mid day of the 10-day average'
alongtrack_gmsl['time']=alongtrack_gmsl.time+(4*24*60*60*10**(9))
alongtrack_gmsl

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data

In [10]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_noerror_simpleweight_nomean_nosicapplied.nc')
alongtrack_gmsl.to_netcdf(fname)

### Random noise error
#### Compute the reference global mean SSH over 20 years 1996-2016

In [11]:
# ECCO_alongtrack_files=[]
# for year in range(1996,2017):
#     ECCO_alongtrack_files_tmp = list(alongtrack_file_dir_randomnoise.glob('*ECCO_V4r4_alongtrack_SSH_*'+str(year)+'*nc'))
#     ECCO_alongtrack_files.extend(ECCO_alongtrack_files_tmp)
# print(np.sort(ECCO_alongtrack_files))

In [12]:
# alongtrack = xr.open_mfdataset(
#         paths=ECCO_alongtrack_files,
#         combine='nested',
#         concat_dim='i',
#         decode_cf=True,
#         compat='override',
#         coords='minimal',
#         chunks={'time': 1}  
#     )  
# alongtrack

In [13]:
# ref_mean_randomnoise=alongtrack.SSH_at_xy.mean(skipna=True).values
# print(ref_mean_randomnoise)

#### Compute GMSL 10-day time series

In [14]:
ECCO_alongtrack_files = list(alongtrack_file_dir_randomnoise.glob('*ECCO_V4r4_alongtrack_SSH_*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

[PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise_nosicapplied/ECCO_V4r4_alongtrack_SSH_1992-09-23_randomnoise.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise_nosicapplied/ECCO_V4r4_alongtrack_SSH_1992-09-24_randomnoise.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise_nosicapplied/ECCO_V4r4_alongtrack_SSH_1992-09-25_randomnoise.nc')
 ...
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise_nosicapplied/ECCO_V4r4_alongtrack_SSH_2017-12-29_randomnoise.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise_nosicapplied/ECCO_V4r4_alongtrack_SSH_2017-12-30_randomnoise.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_randomnoise_nosicapplied/ECCO_V4r4_alongtrack_SSH_2017-12-31_randomnoise.nc')]


In [15]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    w=np.cos(np.deg2rad(alongtrack.lat))
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_randomnoise = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [16]:
alongtrack_gmsl_randomnoise=alongtrack_gmsl_randomnoise.drop('cycle_day')
alongtrack_gmsl_randomnoise=alongtrack_gmsl_randomnoise.drop('delta_days')
alongtrack_gmsl_randomnoise.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_randomnoise.attrs['units'] = 'm'
alongtrack_gmsl_randomnoise.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with random noise'#, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_randomnoise.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_randomnoise['time']=alongtrack_gmsl_randomnoise.time+(4*24*60*60*10**(9))
alongtrack_gmsl_randomnoise

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

In [17]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_randomnoise_simpleweight_nomean_nosicapplied.nc')
alongtrack_gmsl_randomnoise.to_netcdf(fname)

### Missing data
#### Compute the reference global mean SSH over 20 years 1996-2016

In [18]:
# ECCO_alongtrack_files=[]
# for year in range(1996,2017):
#     ECCO_alongtrack_files_tmp = list(alongtrack_file_dir_missingdata.glob('*ECCO_V4r4_alongtrack_SSH_*'+str(year)+'*nc'))
#     ECCO_alongtrack_files.extend(ECCO_alongtrack_files_tmp)
# print(np.sort(ECCO_alongtrack_files))

In [19]:
# alongtrack = xr.open_mfdataset(
#         paths=ECCO_alongtrack_files,
#         combine='nested',
#         concat_dim='i',
#         decode_cf=True,
#         compat='override',
#         coords='minimal',
#         chunks={'time': 1}  
#     )  
# alongtrack

In [20]:
# ref_mean_missingdata=alongtrack.SSH_at_xy.mean(skipna=True).values
# print(ref_mean_missingdata)

#### Compute GMSL 10-day time series

In [21]:
ECCO_alongtrack_files = list(alongtrack_file_dir_missingdata.glob('*ECCO_V4r4_alongtrack_SSH*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
# print(ECCO_alongtrack_files)

In [22]:
alongtrack_gmsl_tmp_missingdata=[]
ind_vec=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano_missingdata=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    w=np.cos(np.deg2rad(alongtrack.lat))
    w=w.where(np.isfinite(SSH_at_xy_ano_missingdata))
    alongtrack_gmsl_tmp_missingdata.append((SSH_at_xy_ano_missingdata*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_missingdata = xr.concat(alongtrack_gmsl_tmp_missingdata, dim='time')

In [23]:
alongtrack_gmsl_missingdata=alongtrack_gmsl_missingdata.drop('cycle_day')
alongtrack_gmsl_missingdata=alongtrack_gmsl_missingdata.drop('delta_days')
alongtrack_gmsl_missingdata.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_missingdata.attrs['units'] = 'm'
alongtrack_gmsl_missingdata.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with missing data'#, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_missingdata.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_missingdata['time']=alongtrack_gmsl_missingdata.time+(4*24*60*60*10**(9))
alongtrack_gmsl_missingdata

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

In [24]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_missingdata_simpleweight_nomean_nosicapplied.nc')
alongtrack_gmsl_missingdata.to_netcdf(fname)

### Orbit error - 2cm
#### Compute the reference global mean SSH over 20 years 1996-2016

#### Compute GMSL 10-day time series

In [25]:
ECCO_alongtrack_files = list(alongtrack_file_dir_orbiterror_2cm.glob('*ECCO_V4r4_alongtrack_SSH_*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

[PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_2cm_nosicapplied/ECCO_V4r4_alongtrack_SSH_1992-09-23_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_2cm_nosicapplied/ECCO_V4r4_alongtrack_SSH_1992-09-24_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_2cm_nosicapplied/ECCO_V4r4_alongtrack_SSH_1992-09-25_orbiterror.nc')
 ...
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_2cm_nosicapplied/ECCO_V4r4_alongtrack_SSH_2017-12-29_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_2cm_nosicapplied/ECCO_V4r4_alongtrack_SSH_2017-12-30_orbiterror.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_orbiterror_2cm_nosicapplied/ECCO_V4r4_alongtrack_SSH_2017-12-31_orbiterror.nc')]


In [26]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    w=np.cos(np.deg2rad(alongtrack.lat))
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_orbiterror_2cm = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [27]:
alongtrack_gmsl_orbiterror_2cm=alongtrack_gmsl_orbiterror_2cm.drop('cycle_day')
alongtrack_gmsl_orbiterror_2cm=alongtrack_gmsl_orbiterror_2cm.drop('delta_days')
alongtrack_gmsl_orbiterror_2cm.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_orbiterror_2cm.attrs['units'] = 'm'
alongtrack_gmsl_orbiterror_2cm.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with 2cm orbit error'#, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_orbiterror_2cm.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_orbiterror_2cm['time']=alongtrack_gmsl_orbiterror_2cm.time+(4*24*60*60*10**(9))
alongtrack_gmsl_orbiterror_2cm

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

In [28]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_orbiterror_2cm_simpleweight_nomean_nosicapplied.nc')
alongtrack_gmsl_orbiterror_2cm.to_netcdf(fname)

### 3 errors: Random noise + missing data + orbit error - 2cm
#### Compute the reference global mean SSH over 20 years 1996-2016

#### Compute GMSL 10-day time series

In [29]:
ECCO_alongtrack_files = list(alongtrack_file_dir_3errors_2cm.glob('*ECCO_V4r4_alongtrack_SSH_*nc'))
ECCO_alongtrack_files=np.sort(ECCO_alongtrack_files)
print(ECCO_alongtrack_files)

[PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_2cm_nosicapplied/ECCO_V4r4_alongtrack_SSH_1992-09-23_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_2cm_nosicapplied/ECCO_V4r4_alongtrack_SSH_1992-09-24_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_2cm_nosicapplied/ECCO_V4r4_alongtrack_SSH_1992-09-25_3errors.nc')
 ...
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_2cm_nosicapplied/ECCO_V4r4_alongtrack_SSH_2017-12-29_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_2cm_nosicapplied/ECCO_V4r4_alongtrack_SSH_2017-12-30_3errors.nc')
 PosixPath('/home/jovyan/Data/ECCO_GMSL/ECCO_V4r4_alongtrack_output_3errors_2cm_nosicapplied/ECCO_V4r4_alongtrack_SSH_2017-12-31_3errors.nc')]


In [30]:
alongtrack_gmsl_tmp=[]
for i in np.arange(0,len(ECCO_alongtrack_files),10):
    ECCO_alongtrack_files_cycle=ECCO_alongtrack_files[i:i+10]
    alongtrack = xr.open_mfdataset(
            paths=ECCO_alongtrack_files_cycle,
            combine='nested',
            concat_dim='i',
            decode_cf=True,
            compat='override',
            coords='minimal',
            chunks={'time': 1}  
        )  
    alongtrack=alongtrack.where((np.abs(alongtrack.lat) <= 66))
    SSH_at_xy_ano=alongtrack.SSH_at_xy
    # SSH_at_xy_ano=alongtrack.SSH_at_xy-ref_mean
    w=np.cos(np.deg2rad(alongtrack.lat))
    w=w.where(np.isfinite(SSH_at_xy_ano))
    alongtrack_gmsl_tmp.append((SSH_at_xy_ano*w).sum(skipna=True)/w.sum(skipna=True))
alongtrack_gmsl_3errors_2cm = xr.concat(alongtrack_gmsl_tmp, dim='time')

In [31]:
alongtrack_gmsl_3errors_2cm=alongtrack_gmsl_3errors_2cm.drop('cycle_day')
alongtrack_gmsl_3errors_2cm=alongtrack_gmsl_3errors_2cm.drop('delta_days')
alongtrack_gmsl_3errors_2cm.name = 'gmsl_ecco_alongtrack'
alongtrack_gmsl_3errors_2cm.attrs['units'] = 'm'
alongtrack_gmsl_3errors_2cm.attrs['summary'] = '10-day GMSL time series computed on the alongtrack ECCO data with 3 errors'#, referenced to a 1996-2016 global mean SSH'
alongtrack_gmsl_3errors_2cm.time.attrs['long_name'] = 'first day of the 10-day average'
alongtrack_gmsl_3errors_2cm['time']=alongtrack_gmsl_3errors_2cm.time+(4*24*60*60*10**(9))
alongtrack_gmsl_3errors_2cm

<xarray.DataArray 'gmsl_ecco_alongtrack' (time: 924)>
dask.array<concatenate, shape=(924,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-09-27 1992-10-07 ... 2018-01-04
    cycle    (time) int64 1 2 3 4 5 6 7 8 9 ... 917 918 919 920 921 922 923 924
Attributes:
    units:    m
    summary:  10-day GMSL time series computed on the alongtrack ECCO data wi...

In [32]:
fname = output_dir / ('GMSL_ECCO_alongtrack_19922017_3errors_2cm_simpleweight_nomean_nosicapplied.nc')
alongtrack_gmsl_3errors_2cm.to_netcdf(fname)